# 실시간 데이터 수집, 청킹 및 증분 학습 시스템


In [1]:
# 실시간 데이터 수집, 청킹 및 증분 학습 시스템

# ========================================
# 1. 필요한 라이브러리 설치
# ========================================
!pip install -q transformers datasets torch accelerate
!pip install -q sentence-transformers
!pip install -q feedparser  # RSS 피드 파싱용
!pip install -q beautifulsoup4  # 웹 스크래핑용
!pip install -q scikit-learn
!pip install -q langchain


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 911.9 kB/s eta 0:00:00


In [ ]:
# ========================================
# 2. 라이브러리 임포트
# ========================================
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    pipeline,
)
from datasets import Dataset as HFDataset, load_dataset
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import time
import threading
import queue
import json
import random
from typing import List, Dict, Tuple
import feedparser
import requests
from bs4 import BeautifulSoup
from collections import deque
import warnings

warnings.filterwarnings("ignore")


# ========================================
# 3. 실시간 데이터 수집 클래스
# ========================================
class RealTimeDataCollector:
    """API 키 없이 공개 소스에서 데이터 수집"""

    def __init__(self, buffer_size=100):
        self.data_buffer = deque(maxlen=buffer_size)
        self.is_collecting = False
        self.sources = {
            "wikipedia": "https://en.wikipedia.org/wiki/Special:Random",
            "arxiv_rss": "http://arxiv.org/rss/cs.AI",
            "news_rss": "http://feeds.bbci.co.uk/news/technology/rss.xml",
            "reddit": "https://www.reddit.com/r/MachineLearning/.json",
        }

    def collect_from_wikipedia(self):
        """Wikipedia 랜덤 페이지에서 텍스트 수집"""
        try:
            response = requests.get(self.sources["wikipedia"], timeout=5)
            soup = BeautifulSoup(response.content, "html.parser")

            # 본문 추출
            content = soup.find("div", {"id": "mw-content-text"})
            if content:
                paragraphs = content.find_all("p", limit=3)
                text = " ".join([p.text for p in paragraphs])
                return {
                    "source": "wikipedia",
                    "text": text[:1000],  # 처음 1000자만
                    "timestamp": datetime.now().isoformat(),
                    "title": soup.find("h1").text if soup.find("h1") else "Unknown",
                }
        except Exception as e:
            print(f"Wikipedia 수집 오류: {e}")
        return None

    def collect_from_arxiv(self):
        """ArXiv RSS 피드에서 논문 초록 수집"""
        try:
            feed = feedparser.parse(self.sources["arxiv_rss"])
            if feed.entries:
                entry = random.choice(feed.entries[:10])
                return {
                    "source": "arxiv",
                    "text": entry.summary[:1000],
                    "timestamp": datetime.now().isoformat(),
                    "title": entry.title,
                }
        except Exception as e:
            print(f"ArXiv 수집 오류: {e}")
        return None

    def collect_from_huggingface_datasets(self):
        """HuggingFace datasets에서 샘플 데이터 수집"""
        try:
            # 다양한 공개 데이터셋에서 랜덤 샘플링
            datasets_list = ["ag_news", "imdb", "squad"]
            chosen_dataset = random.choice(datasets_list)

            if chosen_dataset == "ag_news":
                dataset = load_dataset("ag_news", split="train", streaming=True)
                sample = next(iter(dataset.take(1)))
                return {
                    "source": "ag_news",
                    "text": sample["text"],
                    "timestamp": datetime.now().isoformat(),
                    "label": sample["label"],
                }
            elif chosen_dataset == "imdb":
                dataset = load_dataset("imdb", split="train", streaming=True)
                sample = next(iter(dataset.skip(random.randint(0, 100)).take(1)))
                return {
                    "source": "imdb",
                    "text": sample["text"][:1000],
                    "timestamp": datetime.now().isoformat(),
                    "label": sample["label"],
                }
            elif chosen_dataset == "squad":
                dataset = load_dataset("squad", split="train", streaming=True)
                sample = next(iter(dataset.skip(random.randint(0, 100)).take(1)))
                return {
                    "source": "squad",
                    "text": sample["context"][:1000],
                    "timestamp": datetime.now().isoformat(),
                    "question": sample["question"],
                }
        except Exception as e:
            print(f"HuggingFace datasets 수집 오류: {e}")
        return None

    def simulate_streaming_data(self):
        """스트리밍 데이터 시뮬레이션"""
        templates = [
            "최근 {topic}에 대한 연구가 활발히 진행되고 있다. {detail}",
            "{company}는 새로운 {product}를 발표했다. {feature}",
            "{location}에서 {event}가 개최되었다. {description}",
        ]

        topics = ["AI", "딥러닝", "자연어처리", "컴퓨터비전", "강화학습"]
        companies = ["OpenAI", "Google", "Meta", "Microsoft", "Amazon"]
        products = ["언어모델", "비전모델", "멀티모달시스템", "AI 도구"]

        template = random.choice(templates)
        text = template.format(
            topic=random.choice(topics),
            company=random.choice(companies),
            product=random.choice(),
            location="서울",
            event="AI 컨퍼런스",
            detail="이 기술은 향후 산업 전반에 큰 영향을 미칠 것으로 예상된다.",
            feature="이전 버전보다 성능이 크게 향상되었다.",
            description="전 세계 연구자들이 참여했다.",
        )

        return {
            "source": "simulated",
            "text": text,
            "timestamp": datetime.now().isoformat(),
            "id": f"sim_{random.randint(1000, 9999)}",
        }

    def collect_data_continuously(self):
        """지속적으로 데이터 수집"""
        collectors = [
            self.collect_from_huggingface_datasets,
            self.simulate_streaming_data,
            self.collect_from_wikipedia,
            self.collect_from_arxiv,
        ]

        while self.is_collecting:
            try:
                # 랜덤하게 수집 소스 선택
                collector = random.choice(collectors)
                data = collector()

                if data:
                    self.data_buffer.append(data)
                    print(
                        f"[{datetime.now().strftime('%H:%M:%S')}] "
                        f"데이터 수집: {data['source']} - "
                        f"{len(data['text'])} 글자"
                    )

                # 수집 간격 (1-5초 랜덤)
                time.sleep(random.uniform(1, 5))

            except Exception as e:
                print(f"데이터 수집 오류: {e}")
                time.sleep(2)

    def start_collection(self):
        """백그라운드 데이터 수집 시작"""
        self.is_collecting = True
        self.collection_thread = threading.Thread(
            target=self.collect_data_continuously, daemon=True
        )
        self.collection_thread.start()
        print(" 실시간 데이터 수집 시작")

    def stop_collection(self):
        """데이터 수집 중지"""
        self.is_collecting = False
        print(" 데이터 수집 중지")

    def get_recent_data(self, n=10):
        """최근 수집된 데이터 반환"""
        return list(self.data_buffer)[-n:]


# ========================================
# 4. 동적 청킹 시스템
# ========================================
class DynamicChunkingSystem:
    """수집된 데이터를 동적으로 청킹"""

    def __init__(self, chunk_size=200, overlap=50):
        self.chunk_size = chunk_size
        self.overlap = overlap
        self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
        self.chunked_data = []

    def adaptive_chunking(self, text: str, metadata: Dict) -> List[Dict]:
        """적응형 청킹 - 텍스트 특성에 따라 청크 크기 조정"""

        # 텍스트 특성 분석
        avg_sentence_length = len(text.split(".")) / max(text.count("."), 1)

        # 문장이 긴 경우 청크 크기 증가
        adaptive_size = self.chunk_size
        if avg_sentence_length > 50:
            adaptive_size = int(self.chunk_size * 1.5)

        # 토큰화
        tokens = self.tokenizer.encode(text, add_special_tokens=False)

        chunks = []
        start = 0

        while start < len(tokens):
            end = min(start + adaptive_size, len(tokens))
            chunk_tokens = tokens[start:end]

            chunk_text = self.tokenizer.decode(chunk_tokens, skip_special_tokens=True)

            chunks.append(
                {
                    "text": chunk_text,
                    "source": metadata.get("source", "unknown"),
                    "timestamp": metadata.get("timestamp", datetime.now().isoformat()),
                    "chunk_id": f"{metadata.get('source', 'unk')}_{start}_{end}",
                    "token_count": len(chunk_tokens),
                }
            )

            start = end - self.overlap if end < len(tokens) else end

        return chunks

    def process_new_data(self, data_list: List[Dict]) -> List[Dict]:
        """새로운 데이터를 청킹 처리"""
        all_chunks = []

        for data in data_list:
            if "text" in data and data["text"]:
                chunks = self.adaptive_chunking(data["text"], data)
                all_chunks.extend(chunks)

        self.chunked_data.extend(all_chunks)

        # 메모리 관리 - 최대 1000개 청크 유지
        if len(self.chunked_data) > 1000:
            self.chunked_data = self.chunked_data[-1000:]

        return all_chunks


# ========================================
# 5. 증분 학습 시스템
# ========================================
class IncrementalLearningSystem:
    """새로운 데이터로 모델을 증분 학습"""

    def __init__(self, model_name="distilbert-base-uncased"):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        # 간단한 분류 모델 (예: 텍스트 품질 분류)
        self.model = AutoModelForSequenceClassification.from_pretrained(
            model_name, num_labels=3  # 좋음, 보통, 나쁨
        )

        self.training_history = []
        self.performance_metrics = []

        # 임베딩 모델 (청크 유사도 계산용)
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")

        # 학습 데이터 버퍼
        self.training_buffer = []

    def prepare_training_data(self, chunks: List[Dict]) -> HFDataset:
        """청크를 학습 데이터로 변환"""

        # 임시 레이블 생성 (실제로는 규칙 기반 or 휴리스틱 사용)
        def assign_quality_label(chunk):
            text_len = len(chunk["text"])
            if text_len < 50:
                return 2  # 나쁨
            elif text_len < 150:
                return 1  # 보통
            else:
                return 0  # 좋음

        data = {
            "text": [chunk["text"] for chunk in chunks],
            "labels": [assign_quality_label(chunk) for chunk in chunks],
        }

        return HFDataset.from_dict(data)

    def incremental_train(self, new_chunks: List[Dict]):
        """증분 학습 수행"""
        if len(new_chunks) < 10:
            print("학습에 필요한 최소 데이터 부족 (10개 이상 필요)")
            return

        print(f"\n 증분 학습 시작: {len(new_chunks)}개 청크")

        # 학습 데이터 준비
        dataset = self.prepare_training_data(new_chunks)

        # 토큰화
        def tokenize_function(examples):
            return self.tokenizer(
                examples["text"], padding="max_length", truncation=True, max_length=128
            )

        tokenized_dataset = dataset.map(tokenize_function, batched=True)

        # 학습 설정 (경량화된 설정)
        training_args = TrainingArguments(
            output_dir="./incremental_model",
            num_train_epochs=1,  # 증분 학습은 적은 에폭
            per_device_train_batch_size=8,
            warmup_steps=10,
            weight_decay=0.01,
            logging_steps=10,
            save_steps=100,
            evaluation_strategy="no",
            no_cuda=not torch.cuda.is_available(),
            report_to="none",
        )

        # Trainer 생성 및 학습
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized_dataset,
            tokenizer=self.tokenizer,
        )

        # 학습 수행
        start_time = time.time()
        trainer.train()
        training_time = time.time() - start_time

        # 학습 기록 저장
        self.training_history.append(
            {
                "timestamp": datetime.now().isoformat(),
                "num_samples": len(new_chunks),
                "training_time": training_time,
            }
        )

        print(f" 증분 학습 완료 (소요시간: {training_time:.2f}초)")

    def evaluate_model(self, test_chunks: List[Dict]) -> Dict:
        """모델 성능 평가"""
        if not test_chunks:
            return {}

        # 간단한 평가 - 임베딩 품질 체크
        texts = [chunk["text"] for chunk in test_chunks[:10]]
        embeddings = self.embedder.encode(texts)

        # 임베딩 간 코사인 유사도 계산
        similarities = []
        for i in range(len(embeddings)):
            for j in range(i + 1, len(embeddings)):
                sim = np.dot(embeddings[i], embeddings[j]) / (
                    np.linalg.norm(embeddings[i]) * np.linalg.norm(embeddings[j])
                )
                similarities.append(sim)

        metrics = {
            "avg_similarity": np.mean(similarities) if similarities else 0,
            "std_similarity": np.std(similarities) if similarities else 0,
            "num_chunks_evaluated": len(test_chunks),
            "timestamp": datetime.now().isoformat(),
        }

        self.performance_metrics.append(metrics)
        return metrics


# ========================================
# 6. 자동 업데이트 스케줄러
# ========================================
class AutoUpdateScheduler:
    """주기적으로 데이터 수집, 청킹, 학습을 조율"""

    def __init__(
        self,
        collector: RealTimeDataCollector,
        chunker: DynamicChunkingSystem,
        learner: IncrementalLearningSystem,
    ):
        self.collector = collector
        self.chunker = chunker
        self.learner = learner
        self.is_running = False
        self.update_interval = 30  # 30초마다 업데이트
        self.train_interval = 60  # 60초마다 학습
        self.last_train_time = time.time()

    def update_cycle(self):
        """업데이트 사이클 실행"""
        cycle_count = 0

        while self.is_running:
            cycle_count += 1
            print(f"\n 업데이트 사이클 #{cycle_count} 시작")

            # 1. 최근 수집된 데이터 가져오기
            recent_data = self.collector.get_recent_data(20)

            if recent_data:
                print(f"   {len(recent_data)}개 새 데이터 발견")

                # 2. 데이터 청킹
                new_chunks = self.chunker.process_new_data(recent_data)
                print(f"   {len(new_chunks)}개 청크 생성")

                # 3. 주기적 학습 체크
                current_time = time.time()
                if current_time - self.last_train_time > self.train_interval:
                    if len(new_chunks) >= 10:
                        self.learner.incremental_train(new_chunks)

                        # 평가 수행
                        metrics = self.learner.evaluate_model(new_chunks)
                        print(
                            f"   모델 성능: 평균 유사도 = {metrics.get('avg_similarity', 0):.3f}"
                        )

                        self.last_train_time = current_time
                    else:
                        print(f"  ⏸ 학습 대기 중 (데이터 부족)")

                # 4. 상태 리포트
                self.print_status_report()

            # 대기
            time.sleep(self.update_interval)

    def print_status_report(self):
        """현재 시스템 상태 출력"""
        print(f"\n 시스템 상태 리포트")
        print(f"  - 수집된 데이터: {len(self.collector.data_buffer)}개")
        print(f"  - 총 청크: {len(self.chunker.chunked_data)}개")
        print(f"  - 학습 횟수: {len(self.learner.training_history)}회")

        if self.learner.performance_metrics:
            latest_metric = self.learner.performance_metrics[-1]
            print(f"  - 최근 성능: {latest_metric['avg_similarity']:.3f}")

    def start(self):
        """스케줄러 시작"""
        self.is_running = True
        self.scheduler_thread = threading.Thread(target=self.update_cycle, daemon=True)
        self.scheduler_thread.start()
        print(" 자동 업데이트 스케줄러 시작")

    def stop(self):
        """스케줄러 중지"""
        self.is_running = False
        print(" 자동 업데이트 스케줄러 중지")


# ========================================
# 7. 메인 실행 및 데모
# ========================================
class RealTimeChunkingDemo:
    """전체 시스템 통합 데모"""

    def __init__(self):
        print(" 실시간 청킹 및 학습 시스템 초기화")

        # 컴포넌트 초기화
        self.collector = RealTimeDataCollector(buffer_size=200)
        self.chunker = DynamicChunkingSystem(chunk_size=150, overlap=30)
        self.learner = IncrementalLearningSystem("distilbert-base-uncased")
        self.scheduler = AutoUpdateScheduler(self.collector, self.chunker, self.learner)

        print(" 모든 컴포넌트 초기화 완료")

    def run_demo(self, duration_minutes=3):
        """데모 실행"""
        print(f"\n {duration_minutes}분 동안 데모 실행")
        print("=" * 50)

        # 데이터 수집 시작
        self.collector.start_collection()

        # 자동 업데이트 스케줄러 시작
        self.scheduler.start()

        # 실시간 모니터링
        start_time = time.time()
        duration_seconds = duration_minutes * 60

        try:
            while time.time() - start_time < duration_seconds:
                remaining = duration_seconds - (time.time() - start_time)
                print(f"\r⏱️ 남은 시간: {remaining:.0f}초", end="")
                time.sleep(5)

                # 주기적으로 상세 정보 출력
                if int(remaining) % 30 == 0:
                    print("\n")
                    self.display_dashboard()

        except KeyboardInterrupt:
            print("\n\n 사용자에 의해 중단됨")

        finally:
            # 시스템 종료
            self.shutdown()

    def display_dashboard(self):
        """대시보드 표시"""
        print("\n" + "=" * 60)
        print(" 실시간 대시보드")
        print("=" * 60)

        # 수집 통계
        recent_data = self.collector.get_recent_data(5)
        if recent_data:
            print("\n최근 수집 데이터:")
            for data in recent_data[-3:]:
                print(f"  - [{data['source']}] {data['text'][:50]}...")

        # 청킹 통계
        if self.chunker.chunked_data:
            print(f"\n청킹 통계:")
            print(f"  - 총 청크 수: {len(self.chunker.chunked_data)}")

            # 소스별 분포
            source_counts = {}
            for chunk in self.chunker.chunked_data:
                source = chunk.get("source", "unknown")
                source_counts[source] = source_counts.get(source, 0) + 1

            print(f"  - 소스별 분포: {source_counts}")

        # 학습 통계
        if self.learner.training_history:
            print(f"\n학습 통계:")
            print(f"  - 총 학습 횟수: {len(self.learner.training_history)}")
            last_train = self.learner.training_history[-1]
            print(f"  - 마지막 학습: {last_train['timestamp']}")
            print(f"  - 마지막 학습 샘플: {last_train['num_samples']}개")

        print("=" * 60)

    def shutdown(self):
        """시스템 종료"""
        print("\n\n 시스템 종료 중...")

        # 컴포넌트 정지
        self.collector.stop_collection()
        self.scheduler.stop()

        # 최종 리포트
        self.final_report()

        print(" 시스템 종료 완료")

    def final_report(self):
        """최종 리포트 생성"""
        print("\n" + "=" * 60)
        print(" 최종 리포트")
        print("=" * 60)

        print(f"\n수집 데이터:")
        print(f"  - 총 수집: {len(self.collector.data_buffer)}개")

        print(f"\n청킹 결과:")
        print(f"  - 총 청크: {len(self.chunker.chunked_data)}개")

        if self.chunker.chunked_data:
            avg_chunk_size = np.mean(
                [len(c["text"]) for c in self.chunker.chunked_data]
            )
            print(f"  - 평균 청크 크기: {avg_chunk_size:.1f} 문자")

        print(f"\n학습 결과:")
        print(f"  - 학습 세션: {len(self.learner.training_history)}회")

        if self.learner.performance_metrics:
            avg_performance = np.mean(
                [m["avg_similarity"] for m in self.learner.performance_metrics]
            )
            print(f"  - 평균 성능: {avg_performance:.3f}")

        print("=" * 60)


# ========================================
# 8. 실행 코드
# ========================================
if __name__ == "__main__":

    # 데모 시스템 생성
    demo = RealTimeChunkingDemo()

    # 사용자 입력
    print("\n실행 옵션:")
    print("1. 빠른 데모 (1분)")
    print("2. 표준 데모 (3분)")
    print("3. 확장 데모 (5분)")
    print("4. 커스텀 설정")

    choice = input("\n선택 (1-4): ").strip()

    duration_map = {"1": 1, "2": 3, "3": 5, "4": int(input("실행 시간(분): "))}

    duration = duration_map.get(choice, 3)

    # 데모 실행
    demo.run_demo(duration_minutes=duration)

    print("\n\n 데모 완료! 시스템의 주요 기능:")
    print("  1. 공개 소스에서 실시간 데이터 수집 (Wikipedia, ArXiv, HuggingFace)")
    print("  2. 적응형 청킹으로 텍스트 분할")
    print("  3. 증분 학습으로 모델 지속 개선")
    print("  4. 자동 스케줄링으로 전체 파이프라인 관리")

    # 추가 분석 옵션
    print("\n\n 추가 분석을 원하시나요?")
    if input("상세 분석 보기 (y/n): ").lower() == "y":
        # 청크 분석
        if demo.chunker.chunked_data:
            df = pd.DataFrame(demo.chunker.chunked_data)
            print("\n청크 데이터 통계:")
            print(df.describe())

            # 시간대별 분포
            df["hour"] = pd.to_datetime(df["timestamp"]).dt.hour
            print("\n시간대별 수집 분포:")
            print(df["hour"].value_counts().sort_index())

        # 학습 히스토리 분석
        if demo.learner.training_history:
            train_df = pd.DataFrame(demo.learner.training_history)
            print("\n학습 세션 요약:")
            print(train_df[["timestamp", "num_samples", "training_time"]])

    print("\n 모든 작업이 완료되었습니다!")


    ╔════════════════════════════════════════════════════════╗
    ║   실시간 데이터 수집 및 증분 학습 시스템                 ║
    ║   - 자동 데이터 수집 (API 키 불필요)                    ║
    ║   - 동적 텍스트 청킹                                    ║
    ║   - 주기적 모델 업데이트                                ║
    ╚════════════════════════════════════════════════════════╝
    
 실시간 청킹 및 학습 시스템 초기화


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

 모든 컴포넌트 초기화 완료

실행 옵션:
1. 빠른 데모 (1분)
2. 표준 데모 (3분)
3. 확장 데모 (5분)
4. 커스텀 설정

선택 (1-4): 1
실행 시간(분): 1

 1분 동안 데모 실행
 실시간 데이터 수집 시작

 업데이트 사이클 #1 시작
 자동 업데이트 스케줄러 시작
⏱️ 남은 시간: 60초[14:00:37] 데이터 수집: wikipedia - 1000 글자
[14:00:41] 데이터 수집: arxiv - 599 글자
⏱️ 남은 시간: 55초[14:00:45] 데이터 수집: wikipedia - 1000 글자
⏱️ 남은 시간: 50초[14:00:49] 데이터 수집: wikipedia - 200 글자
⏱️ 남은 시간: 45초[14:00:55] 데이터 수집: wikipedia - 771 글자
[14:00:57] 데이터 수집: simulated - 63 글자
⏱️ 남은 시간: 40초[14:00:59] 데이터 수집: wikipedia - 156 글자


README.md: 0.00B [00:00, ?B/s]

⏱️ 남은 시간: 35초
 업데이트 사이클 #2 시작
   7개 새 데이터 발견
⏱️ 남은 시간: 30초   10개 청크 생성

 시스템 상태 리포트
  - 수집된 데이터: 7개
  - 총 청크: 10개
  - 학습 횟수: 0회
[14:01:10] 데이터 수집: ag_news - 144 글자
⏱️ 남은 시간: 25초[14:01:15] 데이터 수집: arxiv - 1000 글자
⏱️ 남은 시간: 20초[14:01:19] 데이터 수집: wikipedia - 1000 글자
⏱️ 남은 시간: 15초[14:01:23] 데이터 수집: simulated - 63 글자
⏱️ 남은 시간: 10초[14:01:29] 데이터 수집: wikipedia - 208 글자


README.md: 0.00B [00:00, ?B/s]

⏱️ 남은 시간: 5초

 시스템 종료 중...
 데이터 수집 중지
 자동 업데이트 스케줄러 중지

 최종 리포트

수집 데이터:
  - 총 수집: 12개

청킹 결과:
  - 총 청크: 10개
  - 평균 청크 크기: 434.5 문자

학습 결과:
  - 학습 세션: 0회
 시스템 종료 완료


 데모 완료! 시스템의 주요 기능:
  1. 공개 소스에서 실시간 데이터 수집 (Wikipedia, ArXiv, HuggingFace)
  2. 적응형 청킹으로 텍스트 분할
  3. 증분 학습으로 모델 지속 개선
  4. 자동 스케줄링으로 전체 파이프라인 관리


 추가 분석을 원하시나요?
[14:01:39] 데이터 수집: imdb - 528 글자
상세 분석 보기 (y/n): y

청크 데이터 통계:
       token_count
count    10.000000
mean    103.100000
std      43.398541
min      40.000000
25%      67.250000
50%     101.500000
75%     147.500000
max     150.000000

시간대별 수집 분포:
hour
14    10
Name: count, dtype: int64

 모든 작업이 완료되었습니다!
